Может получиться найти модель, которая умеет лучше определять автора текста (0.26 слишком мало) | Done

Хештеги:
Берем берт, вычисляем эмбеддинги для всех предложений, усредняем по хештегам. Для каждого предложения находим эмбеддинг каждого токена. Замену ищем из ближайших по косинусному расстоянию, но также не близких по расстоянию левенштайна.

Посчитать метрики берта на задачах MLM и NSP для оригинальных текстов и анонимизированных.
Посчитать метрики на задаче оценки тональности. Датасет для инсты может Миша предоставить, есть в инете также по твиттеру.

Как оставить упоминания других пользователей? Заменить их в тексте на <MASK> например, потом вставить на место токена упоминание пользователя.

In [1]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')

eng_stopwords = stopwords.words('english')

from nltk.stem import WordNetLemmatizer

from gensim.models import fasttext as ft, Word2Vec
from gensim.test.utils import datapath

import fasttext

from cleantext import clean

from transformers import (
    AutoModel, 
    AutoModelForMaskedLM,
    AutoModelForSeq2SeqLM,
    AutoModelForTokenClassification,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from transformers import pipeline

from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import BinaryClassificationEvaluator

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Embedding
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.callbacks import ModelCheckpoint 

import pytorch_lightning as pl
from pytorch_lightning import Trainer

import pandas as pd
import numpy as np
from scipy import sparse
from tqdm.notebook import tqdm

import re
from itertools import chain, islice
import logging
import os
from collections import Counter
import time

from utils import apply_clean, get_text_and_hashtags

CORES = 10

SEED = 42
TRAIN_DOC_COUNT = 10000
TEST_DOC_COUNT = 1000
AUTHOR_COUNT = 100

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
posts_df = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/vpanov/vk/vk_spb_posts_2019.csv')
posts_df = posts_df.dropna(axis=0)
posts_df.head()

,date,post_id,owner_id,text
0,2019-01-02 18:58:00,11205,84,Крутейшие Неоновые маски. Успей заказать первы...
1,2019-02-03 20:55:00,12084,84,Рекомендую! Отличная вещь. Видеорегистратор Xi...
2,2019-02-04 15:57:00,12086,84,"Крутейший набор для рукоделия. <a href=""/a..."
3,2019-02-05 11:16:00,12087,84,Кошелек игровая консоль PlayStation ретро. <i...
4,2019-02-06 19:17:14,12091,84,Предоставлю отчёт по Гос номеру автомобиля: ДТ...


In [3]:
len(posts_df['owner_id'].unique())

482067

In [4]:
offset = 500
posts_df['owner_id'].value_counts()[offset:offset+AUTHOR_COUNT]

227620       1175
371727932    1174
181072430    1172
1378978      1171
36010941     1170
             ... 
309131878    1031
478126947    1030
176091683    1026
289137251    1026
393151306    1024
Name: owner_id, Length: 100, dtype: int64

In [5]:
most_productive_authors = posts_df['owner_id'].value_counts()[offset:offset+AUTHOR_COUNT].index.values
posts_authors_df = posts_df[posts_df.owner_id.isin(most_productive_authors)].sample(frac=1.0, random_state=SEED)
posts_authors_df.head()

,date,post_id,owner_id,text
989187,2019-10-28 00:00:00,49664,134312775,Ты говоришь лететь за мёдом \nв далёкий космос...
1287802,2019-03-23 00:00:00,26829,161384085,"Смотрите ""Наставление за шаг от смерти! Сохран..."
8066156,2019-04-13 05:31:00,4000,106327572,Я набрал 141800 очков на 2329 уровне. Присоеди...
1777838,2019-03-23 00:00:00,31216,338518394,После этого Рецепта Вы Полюбите Печень. Курина...
502411,2019-09-09 00:00:00,5862,36010941,Ракета НОЛЬ 2609 СССР\nVintage soviet USSR wat...


In [ ]:
posts_authors_df[['only_text', 'hashtags']] = posts_authors_df.apply(get_text_and_hashtags, axis=1, result_type='expand')

In [8]:
posts_authors_df['text_clean'] = posts_authors_df.only_text.apply(lambda x: apply_clean(x, True))

In [15]:
def input_space(text):
    return re.sub(r'([,.!\'\"@<>\\\[\]\(\)#\$%^&\*;:])', ' \1', text)

posts_authors_df['text2'] = posts_authors_df['text_clean'].apply(input_space)

count_threshold = 5

v = posts_authors_df['text2'].str.split().tolist()
c = Counter(chain.from_iterable(v))
posts_authors_df['text_clean'] = [' '.join([j for j in i if c[j] > count_threshold]) for i in v]

In [17]:
max_authors = 50
least_long = 50
max_posts_per_user = 800

long_posts = posts_authors_df[posts_authors_df.text_clean.str.len() > least_long]
authors_posts_count = long_posts.owner_id.value_counts()
authors = authors_posts_count[:max_authors].index.tolist()
min_posts = authors_posts_count.values[max_authors - 1]
median_posts = int(authors_posts_count.median())

train_posts = []
test_posts = []

for i in authors:
    author_i_posts = posts_authors_df[(posts_authors_df.text_clean.str.len() > least_long) & (posts_authors_df.owner_id == i)]
    l = len(author_i_posts)
    train_posts.append(author_i_posts[:min(max_posts_per_user, l - 20)])
    test_posts.append(author_i_posts[l - 20:])

train_posts = pd.concat(train_posts).sample(frac=1.0, random_state=SEED)
test_posts = pd.concat(test_posts).sample(frac=1.0, random_state=SEED)

train_posts.to_csv('/home/jovyan/notebooks/vk/train_posts.csv', index=None)
test_posts.to_csv('/home/jovyan/notebooks/vk/test_posts.csv', index=None)

In [2]:
train_posts = pd.read_csv('/home/jovyan/notebooks/vk/train_posts.csv')
test_posts = pd.read_csv('/home/jovyan/notebooks/vk/test_posts.csv')

# Text anonymization

## SynTF method

In [9]:
USE_SYNSETS = False
TEXT_LENGTH = 50

In [10]:
original_docs = nyc_posts_df.loc[nyc_posts_df['text'].str.len() > 100, 'text'][:1000]

KeyError: 'text'

In [135]:
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = re.sub(r'@\w+', ' ', text.lower())
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    words = text.split()
    words = filter(lambda x: x not in eng_stopwords, words)
    return ' '.join(lemmatizer.lemmatize(x) for x in words)

def get_synonyms(uniq_words):
    all_synonyms = set()
    for word in uniq_words:
        synonyms = wordnet.synsets(word)
        all_synonyms.update(chain.from_iterable([word.lemma_names() for word in synonyms]))
    return all_synonyms

docs = original_docs.apply(preprocess).tolist()
if USE_SYNSETS:
    uniq_words = set(chain.from_iterable([doc.split() for doc in docs]))
    synonyms = ' '.join(get_synonyms(uniq_words))
    docs_with_synonyms = [*docs, synonyms]

In [104]:
tfidf = TfidfVectorizer()

if USE_SYNSETS:
    tfidf.fit(docs_with_synonyms)
else:
    tfidf.fit(docs)
doc_vecs = tfidf.transform(docs)
doc_vecs = normalize(doc_vecs, norm='l1')
words = tfidf.get_feature_names()

In [105]:
len(tfidf.get_feature_names())

7458

In [16]:
ft = fasttext.load_facebook_model(datapath('/mnt/ess_storage/DN_1/storage/home/vpanov/cc.en.300.bin.gz'))

In [72]:
word_vecs = [ft.wv[word] for word in words]
word_similarities = cosine_similarity(word_vecs, word_vecs)
word_similarities.shape

(7458, 7458)

In [73]:
def kgram_overlap(word1, word2, k):
    a = set([word1[i:i+k] for i in range(0, len(word1) - k + 1)])
    b = set([word2[i:i+k] for i in range(0, len(word2) - k + 1)])
    inter = len(a.intersection(b))
    return inter / (len(a) + len(b) - inter)

def score(word1, word2):
    idx1, idx2 = tfidf.vocabulary_[word1], tfidf.vocabulary_[word2]
    return word_similarities[idx1, idx2] - 0.3 * kgram_overlap(word1, word2, 2)

https://programming-dp.com/notebooks/ch9.html

In [114]:
def exponential_gen(x, R, u, sensitivity=1, epsilon=25.4):
    # Calculate the score for each element of R
    scores = [u(x, r) for r in R]
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    # Choose an element from R based on the probabilities
    return np.random.choice(R, 1, p=probabilities)

num = 7000
print(words[num], exponential_gen(words[num], words, score))

unforgettable ['unedited']


In [113]:
def exponential(x, R, u, sensitivity=1, epsilon=25.4):
    # Calculate the score for each element of R
    scores = [u(x, r) for r in R]
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    
    return probabilities

word_replace_probs = []

for word in tqdm(words):
    word_replace_probs.append(exponential(word, words, score))

word_replace_probs = np.array(word_replace_probs)
word_replace_probs.shape

(7458, 7458)

In [146]:
for idx, doc in enumerate(docs[:5]):
    print('--ORIGINAL--')
    print(original_docs[original_docs.index[idx]])
    print('--PREPROCESSED--')
    print(doc)
    print('--GENERATED SEQUENCE (WITHOUT WORD ORDER)--')
    words_count = len(doc.split())
    words_ = np.random.choice(words, words_count, p=doc_vecs[idx].todense().tolist()[0])
    for i in range(words_count):
        word_idx = tfidf.vocabulary_[words_[i]]
        words_[i] = np.random.choice(words, 1, p=word_replace_probs[word_idx])[0]
    print(' '.join(words_))
    print('-'*150)

--ORIGINAL--
Right or Left?

Buffalo Chicken slice on the right and Chicken, Bacon(beef), Ranch on the left from @saucny, all new halal pizza spot in New Hyde Park.
--PREPROCESSED--
right left buffalo chicken slice right chicken bacon beef ranch left new halal pizza spot new hyde park
--GENERATED SEQUENCE (WITHOUT WORD ORDER)--
freshwater bend fish horn lyndhurst cabbage placed rice priestley proof goat rippon bacon inside pork nearby spot though
------------------------------------------------------------------------------------------------------------------------------------------------------
--ORIGINAL--
Poor Cyndy!  This superstar massage therapist had to endure an hour of Amy's special playlist with The Sky treatment to break a 4 day cuckoo migraine.  Wow, this treatment works!  But, Cyndy will never ever want to listen to Pearl Jam, DMB or REM again.
--PREPROCESSED--
poor cyndy superstar massage therapist endure hour amy special playlist sky treatment break 4 day cuckoo migraine 

## ER-AE

In [52]:
model_name = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert = AutoModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
print(tokenizer.encode('I like pigs. And apples.'), tokenizer.decode(tokenizer.encode('I like pigs. And apples.')))
print(tokenizer.encode('i like pigs. and apples.'), tokenizer.decode(tokenizer.encode('i like pigs. and apples.')))
print(tokenizer.encode('i like pigs . and apples .'), tokenizer.decode(tokenizer.encode('i like pigs . and apples .')))

[101, 146, 11850, 24109, 15703, 119, 12689, 72894, 11268, 119, 102] [CLS] I like pigs. And apples. [SEP]
[101, 177, 11850, 24109, 15703, 119, 10111, 72894, 11268, 119, 102] [CLS] i like pigs. and apples. [SEP]
[101, 177, 11850, 24109, 15703, 119, 10111, 72894, 11268, 119, 102] [CLS] i like pigs. and apples. [SEP]


In [54]:
def get_words(doc):
    doc = tokenizer.decode(tokenizer.encode(doc, max_length=128, padding='max_length', truncation=True))
#     doc = tokenizer.decode(tokenizer.encode(doc))
    doc = re.sub(r'([\.,\'’\"\-!\?\(\)])', r' \1 ', doc)
    doc = re.sub('\s', ' ', doc)
    return doc.split()

unique_tokens = set(chain.from_iterable([*train_posts.text_clean.apply(get_words).tolist(),
                                       *test_posts.text_clean.apply(get_words).tolist()]))

token2idx = {token: idx for idx, token in enumerate(unique_tokens)}
idx2token = {idx: token for idx, token in enumerate(unique_tokens)}

In [55]:
BOS_TOKEN_ID = token2idx['[CLS]']
EOS_TOKEN_ID = token2idx['[SEP]']
PAD_TOKEN_ID = token2idx['[PAD]']

VOCAB_SIZE = len(unique_tokens)

In [56]:
del unique_tokens

In [57]:
VOCAB_SIZE

44463

In [10]:
def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)

def get_hidden_states(encoded, token_ids_words, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them."""
    with torch.no_grad():
        output = model(**encoded)
 
    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
    
    res = []
    labels_count = []
    
    for idx, (outp, label) in enumerate(zip(output, token_ids_words)):
        if label is None or token_ids_words[idx - 1] is None or token_ids_words[idx - 1] != token_ids_words[idx]:
            res.append(outp)
            labels_count.append(1)
        else: 
            res[-1] += outp
            labels_count[-1] += 1
    
    res = torch.vstack(res)
    res = res / torch.tensor(labels_count).float().unsqueeze(1)
 
    return res


def get_word_vectors(sent, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt", max_length=128, padding='max_length', truncation=True)
    return get_hidden_states(encoded, encoded.word_ids(), model, layers)


def exmpl(layers=None):
    # Use last four layers by default
    layers = [-4, -3, -2, -1] if layers is None else layers

    sent = train_posts.text_clean[train_posts.index[23]]

    word_embedding = get_word_vectors(sent, tokenizer, bert, layers)

    return word_embedding 

In [11]:
exmpl().shape

torch.Size([121, 768])

In [12]:
def get_word_vectors(sent, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
    
    encoded = tokenizer.encode_plus(sent, return_tensors="pt", max_length=128, padding='max_length', truncation=True)
    
    input_ids = list(map(lambda x: token2idx[x], get_words(sent)))
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    return get_hidden_states(encoded, encoded.word_ids(), model, layers).cpu(), input_ids

def get_embedding(doc, model=bert):
    "Get embedding for each word"
    layers = [-4, -3, -2, -1]
    return get_word_vectors(doc, tokenizer, model, layers)

In [13]:
embeddings = []
input_ids = []

token_embeddings = torch.zeros((VOCAB_SIZE, 768))
token_count = torch.zeros((VOCAB_SIZE,))

for idx, doc in enumerate(train_posts.text_clean):
#     try:
    embedding, ids = get_embedding(doc)
    embeddings.append(embedding)
    input_ids.append(ids)
    token_embeddings.scatter_add_(0, ids[0].unsqueeze(1).expand(embedding.shape), embedding)
    token_count.scatter_add_(0, ids[0], torch.ones_like(ids[0], dtype=torch.float))
#     except Exception as e:
#         print(idx)
#         print(get_words(doc))
#         print(e)

token_embeddings = torch.nan_to_num(torch.div(token_embeddings, token_count.unsqueeze(1).expand(token_embeddings.shape)))
F.normalize(token_embeddings, dim=-1, out=token_embeddings)
del token_count

In [14]:
token_embeddings.shape

torch.Size([44463, 768])

In [15]:
# del token_similarities

In [16]:
token_similarities = torch.zeros((VOCAB_SIZE, VOCAB_SIZE), dtype=torch.float16)

batch_size = 100000
for i in range(VOCAB_SIZE):
    if i % 10000 == 0:
        print(i)
        
    vals = torch.matmul(token_embeddings[i].cuda(), token_embeddings[0:i+1].T.cuda()).cpu()
    torch.clip(vals, max=0.85, out=vals)
    vals = vals.to(torch.float16)
    token_similarities[i, 0:i+1] = vals
    token_similarities[0:i+1, i] = vals

def get_token_sim(i, j, token_similarities=token_similarities):
    return token_similarities[i, j].to(torch.float)

0
10000
20000
30000
40000


In [17]:
pl.utilities.memory.garbage_collection_cuda()
torch.cuda.empty_cache()

In [18]:
with open('checkpoints_vk_distilbert/log.txt', 'a', encoding='utf-8') as f:
    print('token_similarities ok', file=f)

In [19]:
# torch.clip(token_similarities, max=0.85, out=token_similarities)

In [20]:
token_similarities.shape

torch.Size([44463, 44463])

In [21]:
class NYDataset(Dataset):
    def __init__(self, embeddings, input_ids):
        self.embeddings = embeddings
        self.input_ids = list(map(lambda x: x.squeeze(), input_ids))
        
        self.embeddings = nn.utils.rnn.pad_sequence(self.embeddings, batch_first=True, padding_value=0)
        self.input_ids = nn.utils.rnn.pad_sequence(self.input_ids, batch_first=True, padding_value=PAD_TOKEN_ID)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.input_ids[idx].unsqueeze(0)

In [22]:
gen = torch.randn(128, 64, VOCAB_SIZE)
print(gen[:, 0, :].shape)

torch.Size([128, 44463])


In [23]:
gen = F.log_softmax(torch.randn(128, 64, VOCAB_SIZE), -1)
orig = torch.randint(0, VOCAB_SIZE, (64, 128))

gen_doc = gen[:, 0, :]
orig_doc = orig[0, :]

def doc_embed_loss(gen_doc, orig_doc, k=5):
    topk_values, topk_indices = torch.topk(gen_doc, k, dim=-1)
    rand_indices = torch.randint(0, VOCAB_SIZE, (gen_doc.shape[0], k))
    doc_loss = 0
    for i in range(gen_doc.shape[0]):
        rand_values = torch.index_select(gen_doc, -1, rand_indices[i])
        doc_loss += (topk_values * get_token_sim(orig_doc[i].item(), topk_indices[i])).sum()
        doc_loss += (rand_values * get_token_sim(orig_doc[i].item(), rand_indices[i])).sum()
    
    return -doc_loss

doc_embed_loss(gen_doc, orig_doc)  

tensor(900735.2500)

In [24]:
orig_doc[0].item()

5218

In [25]:
gen = torch.randn(64, 128, VOCAB_SIZE)
orig = torch.randint(0, VOCAB_SIZE, (64, 1, 128))

# LOSS FUNCTIONS

def recon_loss(inp, targ):
    "Loss of first stage"
    return F.cross_entropy(inp.view(-1, VOCAB_SIZE), targ.reshape(-1)) # forgot set ignore_index as PAD_TOKEN_ID

def doc_embed_loss(gen_doc, orig_doc, k=5):
    topk_values, topk_indices = torch.topk(gen_doc, k, dim=-1)
    rand_indices = torch.randint(0, VOCAB_SIZE, (gen_doc.shape[0], k))
    doc_loss = 0
    
    for i in range(gen_doc.shape[0]):
        rand_values = torch.index_select(gen_doc, -1, rand_indices[i])
        doc_loss += (topk_values * get_token_sim(orig_doc[i].item(), topk_indices[i])).sum()
        doc_loss += (rand_values * get_token_sim(orig_doc[i].item(), rand_indices[i])).sum()

    return doc_loss.to(torch.float)

def embed_loss(inp, targ, k=5):
    loss = 0
    inp = F.log_softmax(inp, dim=-1)
    for i in range(inp.shape[0]):
        loss += doc_embed_loss(inp[i], targ[i][0], k=k)
    return -loss

def total_loss(inp, targ, alpha=1, beta=0.5, k=5):
    "Loss of second stage"
    return alpha * recon_loss(inp, targ) + beta * embed_loss(inp, targ, k)

print(recon_loss(gen, orig))
print(embed_loss(gen, orig))
print(total_loss(gen, orig))

tensor(11.2026)
tensor(58259076.)
tensor(29129340.)


https://pytorch-lightning.readthedocs.io/en/latest/notebooks/lightning_examples/datamodules.html

In [60]:
def collate_fn(samples):
    x = [sample[0] for sample in samples]
    y = [sample[1].squeeze() for sample in samples]
    
    x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=0.0)
    y = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=PAD_TOKEN_ID)
    
    if x.shape[1] < y.shape[1]:
        y = y[:, :x.shape[1]]
    
#     print(x.shape, y.shape)
    
    return x, y

class LitERAE(pl.LightningModule):
    def __init__(self, data, emb_size=768, hidden_size=512, num_layers=2, act_type=None, learning_rate=1e-3, batch_size=64):

        super().__init__()

        # We hardcode dataset specific stuff here.
        self.data = data

        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        
        self.batch_size = batch_size

        # Build model
        self.gru_1 = nn.GRU(emb_size, hidden_size, num_layers, bidirectional=True, batch_first=True)
        if act_type == None:
            self.act_1 = nn.Identity()
        if act_type == 'ReLU':
            self.act_1 = nn.ReLU()
        self.linear_1 = nn.Linear(hidden_size * 2, emb_size)
        self.gru_2 = nn.GRU(emb_size, hidden_size, num_layers, bidirectional=True, batch_first=True)
        self.linear_2 = nn.Linear(hidden_size * 2, VOCAB_SIZE)

    def forward(self, x):
        x, hidden = self.gru_1(x)
        x = self.act_1(x)
        x = self.linear_1(x)
        x, _ = self.gru_2(x, hidden)
        x = self.act_1(x)
        x = self.linear_2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y)
        
        self.log(f'train_loss', loss)
        self.log(f'avg_train_loss', loss, on_step=False, on_epoch=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y)
        
        self.log(f'val_loss', loss)
        self.log(f'avg_val_loss', loss, on_step=False, on_epoch=True)
        
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        lr_scheduler = {"scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True), "monitor": "avg_val_loss"}
        return {'optimizer': optimizer, 'lr_shceduler': lr_scheduler}

    def on_train_batch_end(self, outputs, batch, batch_idx):
        metrics = self.trainer.callback_metrics
#         logger.info(f'Batch train loss {metrics}')

    def on_train_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'Train loss: {metrics["avg_train_loss"]}')

    def on_validation_batch_end(self, outputs, batch, batch_idx):
        metrics = self.trainer.callback_metrics
#         logger.info(f'Batch validation loss {metrics}')

    def on_validation_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'Val loss: {metrics["avg_val_loss"]}')

    ####################
    # DATA RELATED HOOKS
    ####################

#     def prepare_data(self):
#         self.data = nn.utils.rnn.pad_sequence(self.data)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            train_size = int(0.9 * len(self.data))
            val_size = len(self.data) - train_size
            self.data_train, self.data_val = random_split(self.data, [train_size, val_size], generator=torch.Generator().manual_seed(42))
            self.loss_func = recon_loss
        
        if stage == 'fit_2':
            train_size = int(0.9 * self.data.shape[1])
            val_size = self.data.shape[1] - train_size
            self.data_train, self.data_val = random_split(self.data, [train_size, val_size], generator=torch.Generator().manual_seed(42))
            self.loss_func = total_loss

        # Assign test dataset for use in dataloader(s)
#         if stage == "test" or stage is None:
#             self.data_test
#             self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.data_train, batch_size=self.batch_size, shuffle=True, collate_fn=collate_fn)

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=self.batch_size, shuffle=False, collate_fn=collate_fn)

#     def test_dataloader(self):
#         return DataLoader(self.mnist_test, batch_size=BATCH_SIZE)

In [61]:
chechpoint_path = "checkpoints_vk_distilbert"

In [63]:
BATCH_SIZE = 32

params = {
    'hidden_size': 700,
    'num_layers': 2,
    'learning_rate': 1e-3,
    'act_type': 'ReLU'
}

In [53]:
data = NYDataset(embeddings, input_ids)
model = LitERAE(data, batch_size=BATCH_SIZE, **params)
model.train()

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=chechpoint_path, save_top_k=2, monitor="val_loss")

trainer = Trainer(
    max_epochs=15,
    num_nodes=1,
    num_sanity_val_steps=0,
    callbacks=[checkpoint_callback]
)

trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type   | Params
------------------------------------
0 | gru_1    | GRU    | 15.0 M
1 | act_1    | ReLU   | 0     
2 | linear_1 | Linear | 1.1 M 
3 | gru_2    | GRU    | 15.0 M
4 | linear_2 | Linear | 62.3 M
------------------------------------
93.4 M    Trainable params
0         Non-trainable params
93.4 M    Total params
373.494   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Val loss: 1.845604658126831
Train loss: 2.1288845539093018


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Val loss: 0.4064443111419678
Train loss: 0.04413075000047684


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.


Val loss: 0.4098970592021942
Train loss: 0.042632874101400375


In [54]:
best_model_path = checkpoint_callback.best_model_path

In [55]:
with open(f'{chechpoint_path}/best1.txt', 'w', encoding='utf-8') as f:
    f.write(best_model_path)

In [56]:
del data
del model
del trainer
pl.utilities.memory.garbage_collection_cuda()
# torch.cuda.empty_cache()

In [57]:
!nvidia-smi

Wed Jun 14 14:37:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:5E:00.0 Off |                    0 |
| N/A   64C    P0    38W /  70W |   3078MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [58]:
data = NYDataset(embeddings, input_ids)
model = LitERAE.load_from_checkpoint(checkpoint_path=best_model_path, data=data, **params)

In [59]:
model.stage = 'fit_2'
model.train()

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=chechpoint_path, save_top_k=2, monitor="val_loss")

trainer = Trainer(
    max_epochs=10,
    num_nodes=1,
    num_sanity_val_steps=0,
    callbacks=[checkpoint_callback]
)

trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type   | Params
------------------------------------
0 | gru_1    | GRU    | 15.0 M
1 | act_1    | ReLU   | 0     
2 | linear_1 | Linear | 1.1 M 
3 | gru_2    | GRU    | 15.0 M
4 | linear_2 | Linear | 62.3 M
------------------------------------
93.4 M    Trainable params
0         Non-trainable params
93.4 M    Total params
373.494   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Val loss: 0.369366854429245
Train loss: 0.06973734498023987


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Val loss: 0.38352853059768677
Train loss: 0.01897362992167473


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Val loss: 0.3885502815246582
Train loss: 0.017361585050821304


In [60]:
best_model_path = checkpoint_callback.best_model_path

In [61]:
with open(f'{chechpoint_path}/best2.txt', 'w', encoding='utf-8') as f:
    f.write(best_model_path)

In [62]:
del data
del model
del trainer
# torch.cuda.empty_cache()
pl.utilities.memory.garbage_collection_cuda()

In [63]:
with open('checkpoints_vk_distilbert/best2.txt', 'r', encoding='utf-8') as f:
    best_model_path = f.readlines()[0]

In [64]:
data = NYDataset(embeddings, input_ids)
model = LitERAE.load_from_checkpoint(checkpoint_path=best_model_path, data=data, **params)

In [66]:
def exponential_mechanism(pho, epsilon, delta):
    pho = np.array(pho)
    temp = np.exp(epsilon / (2 * delta) * pho)
    return temp / np.sum(temp)


def predict(model, sent, seed=None):
    rng = np.random.default_rng(seed)
    
    inp = get_embedding(sent)[0].unsqueeze(0).cuda()
    with torch.no_grad():
        logits = model(inp)[0].cpu()
    
    predicted_probs = torch.softmax(logits, dim=-1)
    
    res = []
    
    def build_two_sets(probs, k=5):
        # return lexical set and semantic set
        probs = np.array(probs)
        l_set = rng.choice(probs.shape[0], k, p=probs, replace=True)
        l_set_probs = probs[l_set]

        marks = np.ones(probs.shape[0], dtype=bool)
        marks[l_set] = False

        whole_idxs = np.arange(probs.shape[0])
        s_set = whole_idxs[marks]
        s_set_probs = probs[marks]

        return l_set, s_set, l_set_probs, s_set_probs

    def choose_set(l_set, s_set, l_set_probs, s_set_probs, eps=180):
        probs = [0, 0]
        probs[0] = np.sum(l_set_probs) / (np.sum(l_set_probs) + np.sum(s_set_probs))
        probs[1] = 1 - probs[0]
        probs = exponential_mechanism(probs, eps, 1)
        po = [(l_set, l_set_probs), (s_set, s_set_probs)]
        indxs = [0, 1]
        indx = int(rng.choice(indxs, 1, p=probs))
        return po[indx]

    for probs in predicted_probs:
        # build set
        l_set, s_set, l_set_probs, s_set_probs = build_two_sets(probs, k=5)

        # choose set
        c_set, c_set_probs = choose_set(l_set, s_set, l_set_probs, s_set_probs)

        # choose token
        token_eps = 0.1
        c_set_probs = exponential_mechanism(c_set_probs, token_eps, 1)
        token_idx = int(rng.choice(c_set, 1, p=c_set_probs))

        if token_idx == EOS_TOKEN_ID:
            break
        res.append(token_idx)
#     o_pred = ' '.join([self.params['idx2word'][idx] for idx in res])
#     o_pred = tokenizer.decode(res, skip_special_tokens=True)
    o_pred = ' '.join(idx2token[idx] for idx in res[1:])
    return o_pred

In [66]:
model.eval()

text = test_posts.text_clean[test_posts.index[1]]
print('<Original>:')
print(text)
print('--------------------------------->')
print('<Transformed>:')
print(predict(model, text))

<Original>:
ура  ура  получили новинки линия  22 112 размер  50 52 54 56 58 60 цена  1100 р
--------------------------------->
<Transformed>:
ура ура получили новинки карта 22 150 размер 50 52 54 58 58 60 цена 500 р


In [67]:
model.eval()

test_texts = test_posts.text_clean
for i in range(100, 110):
    text = test_texts[test_posts.index[i]]
    print('<Original>:')
    print(text)
    print('------------------------>')
    print('<Transformed>:')
    print(predict(model, text))
    print('\n------------------------------------------------------------------------\n')

<Original>:
доброе утро  легкого понедельника  осень  утро  дом   горечь  кофе   дождь  тоска  душа  взгляд  компьютер  вздох  надежда  грусть  сомненье  интернет  кнопка   экран  привет  вера  сердце  стук  мгновенье  блеск  глаза  миг  солнце  лучик  одночасье  радость  смех  улыбка  счастье  ника
------------------------>
<Transformed>:
доброе утро легкого располагается осень утро море проруби кофе дождь поляна blue взгляд ольга впечатлений надежду смех солнечного сад dial подарок привет веру дмитрий атмосфера аромат блеск глаза ст солнце dial edition радости страсть [PAD] счастье edition

------------------------------------------------------------------------

<Original>:
estel - салон красоты у вас дома выбрать совместные покупки спб клюкvа  наши раздачи
------------------------>
<Transformed>:
estel - салат красоту у вас 13 выбрать совместные покупки спб клюкvа наши раздачи

------------------------------------------------------------------

### Anonymyze texts

In [58]:
bert = bert.to('cuda')

def get_hidden_states_batch(encoded, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them."""
    
    with torch.no_grad():
        states = model(**encoded).hidden_states
 
    batch_res = []
    
    for i in range(len(states[0])):
        token_ids_words = encoded.word_ids(i)
        output = torch.stack([states[layer][i] for layer in layers]).sum(0).squeeze().cpu()

        res = []
        labels_count = []

        for idx, (outp, label) in enumerate(zip(output, token_ids_words)):
            if label is None or token_ids_words[idx - 1] is None or token_ids_words[idx - 1] != token_ids_words[idx]:
                res.append(outp)
                labels_count.append(1)
            else: 
                res[-1] += outp
                labels_count[-1] += 1

        res = torch.vstack(res)
        res = res / torch.tensor(labels_count).float().unsqueeze(1)
        
        batch_res.append(res)
    batch_res = nn.utils.rnn.pad_sequence(batch_res, batch_first=True, padding_value=0.0)
    return batch_res

def get_word_vectors_batch(sents, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
    
    encoded = tokenizer.batch_encode_plus(sents, return_tensors="pt", max_length=128, padding='max_length', truncation=True)
    
    device = torch.device('cuda')
    encoded = encoded.to(device)

    hidden_states = get_hidden_states_batch(encoded, model, layers)
    return hidden_states

def get_embedding_batch(docs, model=bert):
    "Get embedding for each word"
    layers = [-4, -3, -2, -1]
    return get_word_vectors_batch(docs, tokenizer, model, layers)

def predict_batch(model, sents, seed=None, batch_size=32, verbose=False, print_step=20):
    model.eval()
    rng = np.random.default_rng(seed)
    
    def build_two_sets(probs, k=5):
        # return lexical set and semantic set
        probs = np.array(probs)
        l_set = rng.choice(probs.shape[0], k, p=probs, replace=True)
        l_set_probs = probs[l_set]

        marks = np.ones(probs.shape[0], dtype=bool)
        marks[l_set] = False

        whole_idxs = np.arange(probs.shape[0])
        s_set = whole_idxs[marks]
        s_set_probs = probs[marks]

        return l_set, s_set, l_set_probs, s_set_probs

    def choose_set(l_set, s_set, l_set_probs, s_set_probs, eps=80):
        probs = [0, 0]
        probs[0] = np.sum(l_set_probs) / (np.sum(l_set_probs) + np.sum(s_set_probs))
        probs[1] = 1 - probs[0]
        probs = exponential_mechanism(probs, eps, 1)
        po = [(l_set, l_set_probs), (s_set, s_set_probs)]
        indxs = [0, 1]
        indx = int(rng.choice(indxs, 1, p=probs))
        return po[indx]
    
    o_pred = []
    
    for i in range(0, len(sents), batch_size):
        batch = sents[i:i+batch_size]
        inp = get_embedding_batch(batch).to('cuda')
        with torch.no_grad():
            logits = model(inp).cpu()

        predicted_probs = torch.softmax(logits, dim=-1)
    
        for sent in predicted_probs:
            res = []
            for probs in sent:
                # build set
                l_set, s_set, l_set_probs, s_set_probs = build_two_sets(probs, k=5)

                # choose set
                c_set, c_set_probs = choose_set(l_set, s_set, l_set_probs, s_set_probs)

                # choose token
                token_eps = 0.1
                c_set_probs = exponential_mechanism(c_set_probs, token_eps, 1)
                token_idx = int(rng.choice(c_set, 1, p=c_set_probs))

                if token_idx == EOS_TOKEN_ID:
                    break
                res.append(token_idx)
        #     o_pred = tokenizer.decode(res, skip_special_tokens=True)
            o_pred.append(' '.join(idx2token[idx] for idx in res[1:]))
        
        if verbose and (i % (print_step * batch_size)) == 0:
            print(i)

    return o_pred

In [77]:
train__ = train_posts.text_clean.tolist()
model.eval()
trans_train = predict_batch(model, train__, seed=42, verbose=True)

train_posts['text_clean_anon'] = trans_train
train_posts.to_csv('/home/jovyan/notebooks/vk/train_posts_anon.csv', index=None)

0
640
1280
1920
2560
3200
3840
4480
5120
5760
6400
7040
7680
8320
8960
9600
10240
10880
11520
12160
12800
13440
14080
14720
15360
16000
16640
17280
17920
18560
19200
19840
20480
21120
21760
22400
23040
23680
24320
24960
25600
26240
26880
27520
28160
28800
29440
30080
30720
31360
32000
32640
33280
33920
34560
35200
35840
36480
37120
37760
38400


In [80]:
test__ = test_posts.text_clean.tolist()
model.eval()
trans_test = predict_batch(model, test__, seed=42, verbose=True)

test_posts['text_clean_anon'] = trans_test
test_posts.to_csv('/home/jovyan/notebooks/vk/test_posts_anon.csv', index=None)

In [9]:
train_posts_anon = pd.read_csv('/home/jovyan/notebooks/vk/train_posts_anon.csv')
test_posts_anon = pd.read_csv('/home/jovyan/notebooks/vk/test_posts_anon.csv')

# Hashtags anonymization

## SynTF

In [93]:
docs = nyc_posts_authors_df.loc[nyc_posts_authors_df['hashtags'].str.len() > 50, 'hashtags'][:10000]

In [94]:
docs.values[0]

'#bike #bikes #bikelife #shopride #shoplife #myfavoritebikeshop #bikeclub #bikenyc #bikeny #croton #crotonaqueduct #gravelgrinder #gravel #gravelride #offroad'

In [83]:
from itertools import chain

s = set(chain.from_iterable(docs.str.split()))
len(s)

18078

In [144]:
from sklearn.feature_extraction.text import CountVectorizer

count_model = CountVectorizer(ngram_range=(1,1))
X = count_model.fit_transform(docs)
Xc = (X.T * X)
Xc.setdiag(0)
Xc = Xc.todense()

In [145]:
Xc.sum(), Xc

(5119884,
 matrix([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 5, 5],
         [0, 0, 0, ..., 5, 0, 5],
         [0, 0, 0, ..., 5, 5, 0]]))

In [146]:
Xc.shape

(13364, 13364)

In [125]:
Xc = torch.softmax(torch.from_numpy(Xc).float(), dim=-1)
Xc

tensor([[7.4684e-05, 7.4684e-05, 7.4684e-05,  ..., 7.4684e-05, 7.4684e-05,
         7.4684e-05],
        [7.4582e-05, 7.4582e-05, 7.4582e-05,  ..., 7.4582e-05, 7.4582e-05,
         7.4582e-05],
        [7.3181e-05, 7.3181e-05, 7.3181e-05,  ..., 7.3181e-05, 7.3181e-05,
         7.3181e-05],
        ...,
        [3.0429e-07, 3.0429e-07, 3.0429e-07,  ..., 3.0429e-07, 4.5161e-05,
         4.5161e-05],
        [3.0429e-07, 3.0429e-07, 3.0429e-07,  ..., 4.5161e-05, 3.0429e-07,
         4.5161e-05],
        [3.0429e-07, 3.0429e-07, 3.0429e-07,  ..., 4.5161e-05, 4.5161e-05,
         3.0429e-07]])

In [126]:
Xc[0].sum(), Xc[0, 0]

(tensor(1.0000), tensor(7.4684e-05))

In [84]:
tfidf = TfidfVectorizer()

tfidf.fit(docs)
doc_vecs = tfidf.transform(docs)
doc_vecs = normalize(doc_vecs, norm='l1')
words = tfidf.get_feature_names()

In [85]:
len(tfidf.get_feature_names())

18063

In [89]:
ft = fasttext.load_facebook_model(datapath('/mnt/ess_storage/DN_1/storage/home/vpanov/cc.en.300.bin.gz'))

(1405975, 5131770)

In [160]:
Xc[count_model.vocabulary_['newyork']]

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [161]:
word_vecs = [Xc[count_model.vocabulary_[word]].tolist()[0] for word in words]
word_similarities = cosine_similarity(word_vecs, word_vecs)
word_similarities.shape

(13364, 13364)

In [162]:
def kgram_overlap(word1, word2, k):
    a = set([word1[i:i+k] for i in range(0, len(word1) - k + 1)])
    b = set([word2[i:i+k] for i in range(0, len(word2) - k + 1)])
    inter = len(a.intersection(b))
    return inter / (len(a) + len(b) - inter)

def score(word1, word2):
    idx1, idx2 = tfidf.vocabulary_[word1], tfidf.vocabulary_[word2]
    return word_similarities[idx1, idx2] - 0.3 * kgram_overlap(word1, word2, 2)

In [163]:
def exponential_gen(x, R, u, sensitivity=1, epsilon=25.4):
    # Calculate the score for each element of R
    scores = [u(x, r) for r in R]
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    # Choose an element from R based on the probabilities
    return np.random.choice(R, 1, p=probabilities)

num = 7000
print(words[num], exponential_gen(words[num], words, score))

likes4like ['turkiye']


In [164]:
def exponential(x, R, u, sensitivity=1, epsilon=25.4):
    # Calculate the score for each element of R
    scores = [u(x, r) for r in R]
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    
    return probabilities

word_replace_probs = []

for idx, word in enumerate(words):
    if idx % 1000 == 0:
        print(idx)
    word_replace_probs.append(exponential(word, words, score))

word_replace_probs = np.array(word_replace_probs)
word_replace_probs.shape

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300


(13364, 13364)

In [166]:
for idx, doc in enumerate(docs[:5]):
    print('--ORIGINAL--')
    print(doc)
    print('--GENERATED SEQUENCE (WITHOUT WORD ORDER)--')
    words_count = len(doc.split())
    words_ = np.random.choice(words, words_count, p=doc_vecs[idx].todense().tolist()[0])
    for i in range(words_count):
        word_idx = tfidf.vocabulary_[words_[i]]
        words_[i] = np.random.choice(words, 1, p=word_replace_probs[word_idx])[0]
    print(' '.join(words_))
    print('-'*150)

--ORIGINAL--
#EATDRINKPARTY #miercolesplayero #bronx #BottleSpecials #bestiakitchenbx #LaBestiaDelBronx #HappyHour #PartyPeople #FoodPorn #salsa #playero #retro #4thofjulyparty #preindependenceday #preindependencedayparty
--GENERATED SEQUENCE (WITHOUT WORD ORDER)--
gaynightlife nbafinals2019 bachata miercolesplayero eatdrinkparty nbafinals2019 santiago 4thofjulyparty caucau 4thofjulyparty lentejitas nycdrinks ericktorres latino 4thofjulyparty
------------------------------------------------------------------------------------------------------------------------------------------------------
--ORIGINAL--
#914 #newrochelle #newrochelleny #ionacollege #westchester #westchesterny #westchestereats #westchesternyeats #westchestercountyny #westchestercounty #westchesterfood #westchesterfoodie #tuckahoeny #tuckahoe #eastchester #eastchesterny #larchmont #larchmontny #larchmontvillage #pelhamny #yonkers #yonkersny #bronxville #bronxvilleny #burgersandbeer #bestwings
--GENERATED SEQUENCE (WITHOU

# Hashtags magic

In [133]:
docs = nyc_posts_authors_df.loc[nyc_posts_authors_df['hashtags'].str.len() > 50, 'hashtags'][:20000]

In [134]:
hashtags = docs.str.split().tolist()

w2v_model = Word2Vec(min_count=20,
                     window=5,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=10)
w2v_model.build_vocab(hashtags, progress_per=1000)
w2v_model.train(hashtags, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(3536335, 10212810)

In [135]:
w2v_model.wv.most_similar('#newyork')

[('#tagforlikes', 0.747814416885376),
 ('#wedding', 0.7455034255981445),
 ('#like', 0.7374163866043091),
 ('#barmitzvah', 0.7349600791931152),
 ('#aniversary', 0.7308018803596497),
 ('#selfie', 0.7256142497062683),
 ('#foody', 0.7172373533248901),
 ('#picoftheday', 0.7157788276672363),
 ('#restaurants', 0.7054566740989685),
 ('#happy', 0.7051026225090027)]

In [136]:
w2v_model.wv.most_similar('#wedding')

[('#birthday', 0.9645171165466309),
 ('#aniversary', 0.9628538489341736),
 ('#restaurants', 0.9571098685264587),
 ('#barmitzvah', 0.954374372959137),
 ('#tagforlikes', 0.932794451713562),
 ('#sushi', 0.9251395463943481),
 ('#foody', 0.921452522277832),
 ('#like', 0.9046717882156372),
 ('#selfie', 0.885693371295929),
 ('#appetizer', 0.8824301958084106)]

In [137]:
# https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/

def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

#     printDistances(distances, len(token1), len(token2))
    return distances[len(token1)][len(token2)]

def kgram_overlap(word1, word2, k):
    a = set([word1[i:i+k] for i in range(0, len(word1) - k + 1)])
    b = set([word2[i:i+k] for i in range(0, len(word2) - k + 1)])
    inter = len(a.intersection(b))
    return inter / (len(a) + len(b) - inter + 1)

def score(word1, word2):
    return w2v_model.wv.similarity(word1, word2) - 0.5 * kgram_overlap(word1, word2, 3)

In [142]:
# def get_replace(word, k=5, seed=None):
#     repls = [(repl, score(word, repl)) for repl, _ in w2v_model.wv.most_similar(word, topn=k)]
#     repls.sort(key=lambda x: x[1], reverse=True)
#     rng = np.random.default_rng(seed)
#     return rng.choice(repls, p=[score for _, score in repls])[0]

# get_replace('#newyork')

words = list(w2v_model.wv.key_to_index.keys())

def exponential_gen(x, R, u, sensitivity=1, epsilon=25.4, seed=None):
    rng = np.random.default_rng(seed)
    
    # Calculate the score for each element of R
    scores = [u(x, r) for r in R]
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    # Choose an element from R based on the probabilities
    return rng.choice(R, 1, p=probabilities)

word = '#newyork'
print(word, exponential_gen(word, words, score))

#newyork ['#happy']


In [143]:
def exponential(x, R, u, sensitivity=1, epsilon=25.4, seed=None):
    rng = np.random.default_rng(seed)
    # Calculate the score for each element of R
    scores = [u(x, r) for r in R]
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    
    return probabilities

word_replace_probs = []

for idx, word in enumerate(words):
    if idx % 1000 == 0:
        print(idx)
    word_replace_probs.append(exponential(word, words, score))

word_replace_probs = np.array(word_replace_probs)
word_replace_probs.shape

0
1000
2000


(2242, 2242)

In [145]:
rng = np.random.default_rng(0)

for idx, doc in enumerate(docs[:5]):
    print('--ORIGINAL--')
    print(doc)
    print('--GENERATED SEQUENCE (WITHOUT WORD ORDER)--')
    words_ = doc.split()
    for i in range(len(words_)):
        if rng.random() < 0.5 and words_[i] in w2v_model.wv:
            word_idx = w2v_model.wv.key_to_index[words_[i]]
            words_[i] = rng.choice(words, 1, p=word_replace_probs[word_idx])[0]
    print(' '.join(words_))
    print('-'*100)

--ORIGINAL--
#bike #bikes #bikelife #shopride #shoplife #myfavoritebikeshop #bikeclub #bikenyc #bikeny #croton #crotonaqueduct #gravelgrinder #gravel #gravelride #offroad
--GENERATED SEQUENCE (WITHOUT WORD ORDER)--
#bike #cycling #movesale #shopride #shoplife #myfavoritebikeshop #bikeclub #bikenyc #bikeny #croton #crotonaqueduct #gravelgrinder #gravel #gravelride #offroad
----------------------------------------------------------------------------------------------------
--ORIGINAL--
#76ers #raptors #blazers #nuggets #labatt #genesse #molson #nba #nbaplayoffs #nba2019 #basketball #hoops #nbafirstround #beerandhoops #brooklynsportsbar
--GENERATED SEQUENCE (WITHOUT WORD ORDER)--
#76ers #warriors #blazers #nuggets #labatt #genesse #molson #nbaleaguepass #nbaplayoffs #nba2019 #basketball #hoops #nbafirstround #nba2019 #brooklynsportsbar
----------------------------------------------------------------------------------------------------
--ORIGINAL--
#comicartsbrooklyn #prattinstitute #giant

# Attacker

https://github.com/gkhayes/author_attribution CNN AA model

https://github.com/yunitata/continuous-n-gram-AA code for CNN AA experiments reproduce

## CNN n-gram

In [10]:
text_train = tokenizer.batch_decode(tokenizer(train_posts.text_clean.tolist(), return_tensors="pt", max_length=128, padding='max_length', truncation=True).input_ids, skip_special_tokens=True)
text_test = tokenizer.batch_decode(tokenizer(test_posts.text_clean.tolist(), return_tensors="pt", max_length=128, padding='max_length', truncation=True).input_ids, skip_special_tokens=True)

In [11]:
def create_n_grams(excerpt_list, n, vocab_size, seq_size):
    """Create a list of n-gram sequences
    
    Args:
    excerpt_list: list of strings. List of normalized text excerpts.
    n: int. Length of n-grams.
    vocab_size: int. Size of n-gram vocab (used in one-hot encoding)
    seq_size: int. Size of n-gram sequences
    
    Returns:
    n_gram_array: array. Numpy array of one-hot encoded n-grams.
    """
    n_gram_list = []

    for excerpt in excerpt_list:
        # Remove spaces
        excerpt = excerpt.replace(" ", "")

        # Extract n-grams
        n_grams = [excerpt[i:i + n] for i in range(len(excerpt) - n + 1)]

        # Convert to a single string with spaces between n-grams
        new_string = " ".join(n_grams)

        # One hot encode
        hot = one_hot(new_string, round(vocab_size*1.3))

        # Pad hot if necessary
        hot_len = len(hot)
        if hot_len >= seq_size:
            hot = hot[0:seq_size]
        else:
            diff = seq_size - hot_len
            extra = [0]*diff
            hot = hot + extra

        n_gram_list.append(hot)
    
    n_gram_array = np.array(n_gram_list)
    
    return n_gram_array

In [12]:
def get_vocab_size(excerpt_list, n, seq_size):
    """Calculate size of n-gram vocab
    
    Args:
    excerpt_list: list of strings. List of normalized text excerpts.
    n: int. Length of n-grams.
    seq_size: int. Size of n-gram sequences
    
    Returns:
    vocab_size: int. Size of n-gram vocab.
    """
    n_gram_list = []

    for excerpt in excerpt_list:
        # Remove spaces
        excerpt = excerpt.replace(" ", "")

        # Extract n-grams           
        n_grams = [excerpt[i:i + n] for i in range(len(excerpt) - n + 1)]

        # Create list of n-grams
        gram_len = len(n_grams)
        if gram_len >= seq_size:
            n_grams = n_grams[0:seq_size]
        else:
            diff = seq_size - gram_len
            extra = [0]*diff
            n_grams = n_grams + extra
        
        n_gram_list.append(n_grams)
    
    # Flatten n-gram list
    n_gram_list = list(np.array(n_gram_list).flat)
    
    # Calculate vocab size
    n_gram_cnt = Counter(n_gram_list)
    vocab_size = len(n_gram_cnt)
    
    return vocab_size

In [14]:
i = 3
vocab_size = get_vocab_size(text_train, i, 128)
print('Vocab size for n =', i, 'is:', vocab_size)

Vocab size for n = 3 is: 42643


In [15]:
# Create n-gram lists
gram3_train = create_n_grams(text_train, 3, vocab_size, 128)
gram3_test = create_n_grams(text_test, 3, vocab_size, 128)

print(np.shape(gram3_train))
print(np.shape(gram3_test))

(39035, 128)
(1000, 128)


In [16]:
max_3gram = np.max(gram3_train)

print('Maximum encoding value for 3-grams is: ', max_3gram)

Maximum encoding value for 3-grams is:  55434


In [17]:
# Define model architecture in keras
# Code reference: https://github.com/gkhayes/author_attribution
def define_model(input_len, output_size, vocab_size, embedding_dim, verbose = True,
                drop_out_pct = 0.25, conv_filters = 500, activation_fn = 'relu', pool_size = 2, learning = 0.0001):
    """Define n-gram CNN
    
    Args:
    input_len: int. Length of input sequences.
    output_size: int. Number of output classes.
    vocab_size: int. Maximum value of n-gram encoding.
    embedding_dim: int. Size of embedding layer.
    verbose: bool. Whether or not to print model summary.
    drop_out_pct: float. Drop-out rate.
    conv_filters: int. Number of filters in the conv layer.
    activation_fn: string. Activation function to use in the convolutional layer.
    pool_size: int. Pool size for the max pooling layer.
    learning: float. Learning rate for the model optimizer.
    
    Returns:
    model: keras model object. 
    """
    # Channel 1
    inputs1 = Input(shape = (input_len,))
    embedding1 = Embedding(vocab_size, embedding_dim)(inputs1)
    drop1 = Dropout(drop_out_pct)(embedding1)
    conv1 = Conv1D(filters = conv_filters, kernel_size = 3, activation = activation_fn)(drop1)
    pool1 = MaxPooling1D(pool_size = pool_size)(conv1)
    flat1 = Flatten()(pool1)
    
    # Channel 2
    inputs2 = Input(shape = (input_len,))
    embedding2 = Embedding(vocab_size, embedding_dim)(inputs2)
    drop2 = Dropout(drop_out_pct)(embedding2)
    conv2 = Conv1D(filters = conv_filters, kernel_size = 4, activation = activation_fn)(drop2)
    pool2 = MaxPooling1D(pool_size = pool_size)(conv2)
    flat2 = Flatten()(pool2)

    # Channel 3
    inputs3 = Input(shape = (input_len,))
    embedding3= Embedding(vocab_size, embedding_dim)(inputs3)
    drop3 = Dropout(drop_out_pct)(embedding3)
    conv3 = Conv1D(filters = conv_filters, kernel_size = 5, activation = activation_fn)(drop3)
    pool3 = MaxPooling1D(pool_size = pool_size)(conv3)
    flat3 = Flatten()(pool3)
    
    # Merge channels
    merged = Concatenate()([flat1, flat2, flat3])
    
    # Create output layer
    output = Dense(output_size, activation = 'softmax')(merged)
    
    # Create model
    model = Model(inputs = [inputs1, inputs2, inputs3], outputs = output)
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer = Adam(lr = learning), metrics=['accuracy'])
    
    if verbose:
        print(model.summary())
        
    return model

In [20]:
lb = preprocessing.LabelEncoder()

author_train = lb.fit_transform(train_posts.owner_id.values)
author_train_hot = pd.get_dummies(author_train).values
author_test = lb.transform(test_posts.owner_id.values)

In [22]:
# Create the 3-gram model
gram3_model = define_model(128, len(train_posts.owner_id.unique()), max_3gram + 1, 600)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 600)     33261000    ['input_1[0][0]']                
                                                                                              

In [23]:
tf.config.run_functions_eagerly(True)

In [26]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

Num GPUs Available:  0
2.12.0


In [ ]:
gram3_model.fit([gram3_train, gram3_train, gram3_train], author_train_hot, epochs=15, batch_size=32, 
                verbose = 1, validation_split = 0.2)

In [ ]:
# Fit and evaluate Model 1 (3-gram CNN)

# t0 = time.time()

# # Fit model
# model1 = define_model(128, len(train_posts.authorid.unique()), max_3gram + 1, 300)
# model1.fit([gram3_train, gram3_train, gram3_train], author_train_hot, epochs=10, batch_size=32, 
#            verbose = 1, validation_split = 0.2)
t1 = time.time()

# Predict values for test set
author_pred1 = gram3_model.predict([gram3_test, gram3_test, gram3_test]).argmax(-1)

t2 = time.time()

# Evaluate
accuracy = balanced_accuracy_score(author_test, author_pred1)
precision, recall, f1, support = score(author_test, author_pred1, average='weighted')
confusion = confusion_matrix(author_test, author_pred1)
    
print("Accuracy:", accuracy)
print("Ave. Precision:", precision)
print("Ave. Recall:", recall)
print("Ave. F1 Score:", f1)
# print("Training Time:", (t1 - t0), "seconds")
print("Prediction Time:", (t2 - t1), "seconds")
print("Confusion Matrix:\n", confusion)

with open('/home/jovyan/notebooks/vk/attacker_vk.txt', 'w') as f:
    print("Accuracy:", accuracy, file=f)
    print("Ave. Precision:", precision, file=f)
    print("Ave. Recall:", recall, file=f)
    print("Ave. F1 Score:", f1, file=f)
    print("Confusion Matrix:\n", confusion, file=f)

In [ ]:
trans_train = train_texts_anon['text_clean_anon']
trans_test = test_texts_anon['text_clean_anon']

In [ ]:
trans_vocab_size = get_vocab_size(trans_train, 3, 128)
trans_gram3_train = create_n_grams(trans_train, 3, trans_vocab_size, 128)
trans_max_3gram = np.max(trans_gram3_train)

In [ ]:
# Create the 3-gram model
trans_gram3_model = define_model(128, len(train_posts.authorid.unique()), trans_max_3gram + 1, 600)
trans_gram3_model.fit([trans_gram3_train, trans_gram3_train, trans_gram3_train], author_train_hot, epochs=15, batch_size=32, 
                verbose = 1, validation_split = 0.2)

In [ ]:
true_values = author_test

In [ ]:
preds = gram3_model.predict([gram3_test, gram3_test, gram3_test]).argmax(-1)

trans_vocab_size = get_vocab_size(trans_train, 3, 128)
trans_gram3_test = create_n_grams(trans_test, 3, trans_vocab_size, 128)
trans_preds = trans_gram3_model.predict([trans_gram3_test, trans_gram3_test, trans_gram3_test]).argmax(-1)

vocab_size = get_vocab_size(text_train, 3, 128)
orig_trans_gram3_test = create_n_grams(trans_test, 3, vocab_size, 128)
orig_trans_preds = gram3_model.predict([trans_gram3_test, trans_gram3_test, trans_gram3_test]).argmax(-1)

print('Weighted scores')
print('F1 score for original text:', f1_score(true_values, preds, average='weighted'))
print('F1 score for transformed text with transformed model:', f1_score(true_values, trans_preds, average='weighted'))
print('F1 score for transformed text with orig model:', f1_score(true_values, orig_trans_preds, average='weighted'))
print()
print('Macro averaged scores')
print('F1 score for original text:', f1_score(true_values, preds, average='macro'))
print('F1 score for transformed text with transformed model:', f1_score(true_values, trans_preds, average='macro'))
print('F1 score for transformed text with orig model:', f1_score(true_values, orig_trans_preds, average='macro'))

with open('/home/jovyan/notebooks/vk/attacker_tests_vk.txt', 'w') as f:
    print('Weighted scores', file=f)
    print('F1 score for original text:', f1_score(true_values, preds, average='weighted'), file=f)
    print('F1 score for transformed text with transformed model:', f1_score(true_values, trans_preds, average='weighted'), file=f)
    print('F1 score for transformed text with orig model:', f1_score(true_values, orig_trans_preds, average='weighted'), file=f)
    print('', file=f)
    print('Macro averaged scores', file=f)
    print('F1 score for original text:', f1_score(true_values, preds, average='macro'), file=f)
    print('F1 score for transformed text with transformed model:', f1_score(true_values, trans_preds, average='macro'), file=f)
    print('F1 score for transformed text with orig model:', f1_score(true_values, orig_trans_preds, average='macro'), file=f)

## BERT sentiment

In [64]:
with open('checkpoints_vk_distilbert/best2.txt', 'r', encoding='utf-8') as f:
    best_model_path = f.readlines()[0]
data = None
model = LitERAE.load_from_checkpoint(checkpoint_path=best_model_path, data=data, **params)

In [40]:
from datetime import datetime

vk_sentiment_pos_df = pd.read_csv('/home/jovyan/notebooks/twitter/positive.csv', usecols=['tdate', 'tname', 'ttext', 'ttype'])
vk_sentiment_pos_df['tdate'] = vk_sentiment_pos_df['tdate'].apply(datetime.fromtimestamp)

vk_sentiment_neg_df = pd.read_csv('/home/jovyan/notebooks/twitter/negative.csv', usecols=['tdate', 'tname', 'ttext', 'ttype'])
vk_sentiment_neg_df['tdate'] = vk_sentiment_neg_df['tdate'].apply(datetime.fromtimestamp)

vk_sentiment_df = pd.concat([vk_sentiment_pos_df, vk_sentiment_neg_df])

In [42]:
subset_cond = (vk_sentiment_df['tdate'] >= datetime(2013, 12, 6)) & (vk_sentiment_df['tdate'] < datetime(2013, 12, 7))
vk_sentiment_df = vk_sentiment_df[subset_cond]

In [44]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
vk_sentiment_df['target'] = lb.fit_transform(vk_sentiment_df['ttype'])

In [50]:
vk_sentiment_df['text'] = vk_sentiment_df['ttext'].apply(apply_clean)
vk_sentiment_df[['only_text', 'hashtags']] = vk_sentiment_df.apply(get_text_and_hashtags, axis=1, result_type='expand')

In [67]:
vk_sentiment_df['anonymized_text'] = predict_batch(model, vk_sentiment_df['only_text'].tolist(), seed=42)

In [68]:
train_sentiment, test_sentiment = train_test_split(vk_sentiment_df, test_size=0.1, random_state=42)

In [69]:
train_sentiment.to_csv('/home/jovyan/notebooks/twitter/train_sentiment.csv')
test_sentiment.to_csv('/home/jovyan/notebooks/twitter/test_sentiment.csv')

In [47]:
train_sentiment = pd.read_csv('/home/jovyan/notebooks/twitter/train_sentiment.csv')
test_sentiment = pd.read_csv('/home/jovyan/notebooks/twitter/test_sentiment.csv')

In [70]:
sentiment_model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'

num_labels = train_sentiment.target.nunique()

sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)

def sentiment_model_init(model_name=sentiment_model_name, num_labels=num_labels):
    return AutoModelForSequenceClassification.from_pretrained(sentiment_model_name, num_labels=num_labels, ignore_mismatched_sizes=True)

In [71]:
class SentimentDataset(Dataset):
    def __init__(self, texts, targets, tokenizer):
        self.inputs = tokenizer(texts, return_tensors="pt", max_length=128, padding='max_length', truncation=True)
        self.outputs = targets

    def __len__(self):
        return len(self.outputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'token_type_ids': self.inputs['token_type_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'targets': self.outputs[idx]
        }

In [76]:
BERT_BATCH_SIZE = 8

class LitSentiment(pl.LightningModule):
    def __init__(self, model_init, train, test, learning_rate=1e-4):

        super().__init__()

        # We hardcode dataset specific stuff here.
        self.train_dataset = train
        self.test_dataset = test

        self.learning_rate = learning_rate
        self.model = model_init()

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        x = {a: b for a, b in batch.items() if a != 'targets'}
        y = batch['targets']
        outputs = self(**x)
        logits = outputs.logits
        loss = self.loss_func(logits, y)
        
        f1 = f1_score(y.cpu(), logits.cpu().argmax(dim=-1), average='macro')
        
        self.log(f'train_loss', loss)
        self.log(f'avg_train_loss', loss, on_step=False, on_epoch=True)
        self.log(f'train_f1', f1)
        self.log(f'avg_train_f1', f1, on_step=False, on_epoch=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x = {a: b for a, b in batch.items() if a != 'targets'}
        y = batch['targets']
        outputs = self(**x)
        logits = outputs.logits
        loss = self.loss_func(logits, y)
        
        f1 = f1_score(y.cpu(), logits.cpu().argmax(dim=-1), average='macro')
        
        self.log(f'val_loss', loss)
        self.log(f'avg_val_loss', loss, on_step=False, on_epoch=True)
        self.log(f'val_f1', f1)
        self.log(f'avg_val_f1', f1, on_step=False, on_epoch=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        x = {a: b for a, b in batch.items() if a != 'targets'}
        y = batch['targets']
        outputs = self(**x)
        logits = outputs.logits
        loss = self.loss_func(logits, y)
        
        f1 = f1_score(y.cpu(), logits.cpu().argmax(dim=-1), average='macro')
        
        self.log(f'test_loss', loss)
        self.log(f'avg_test_loss', loss, on_step=False, on_epoch=True)
        self.log(f'test_f1', f1)
        self.log(f'avg_test_f1', f1, on_step=False, on_epoch=True)
        
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    def on_train_batch_end(self, outputs, batch, batch_idx):
        metrics = self.trainer.callback_metrics
#         logger.info(f'Batch train loss {metrics}')

    def on_train_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'Train loss: {metrics["avg_train_loss"]}')
        print(f'Train f1: {metrics["avg_train_f1"]}')

    def on_validation_batch_end(self, outputs, batch, batch_idx):
        metrics = self.trainer.callback_metrics
#         logger.info(f'Batch validation loss {metrics}')

    def on_validation_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'Val loss: {metrics["avg_val_loss"]}')
        print(f'Val f1: {metrics["avg_val_f1"]}')

    def on_test_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'Test loss: {metrics["avg_test_loss"]}')
        print(f'Test f1: {metrics["avg_test_f1"]}')

    ####################
    # DATA RELATED HOOKS
    ####################

#     def prepare_data(self):
#         self.data = nn.utils.rnn.pad_sequence(self.data)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            train_size = int(0.9 * len(self.train_dataset))
            val_size = len(self.train_dataset) - train_size
            self.data_train, self.data_val = random_split(self.train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.data_test = self.test_dataset

        self.loss_func = nn.CrossEntropyLoss() # forgot to add ignore_index for BERT

    def train_dataloader(self):
        return DataLoader(self.data_train, batch_size=BERT_BATCH_SIZE, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=BERT_BATCH_SIZE, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.data_test, batch_size=BERT_BATCH_SIZE, shuffle=False)

In [77]:
chechpoint_path_sentiment = "checkpoints_ru_sentiment"

In [78]:
train_dataset = SentimentDataset(train_sentiment.text.tolist(), train_sentiment.target.tolist(), sentiment_tokenizer)
test_dataset = SentimentDataset(test_sentiment.text.tolist(), test_sentiment.target.tolist(), sentiment_tokenizer)
sentiment_pl = LitSentiment(sentiment_model_init, train_dataset, test_dataset, 2e-5)
sentiment_pl.train()

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=chechpoint_path_sentiment, save_top_k=2, monitor="val_loss")

trainer = Trainer(
    max_epochs=5,
    num_nodes=1,
    num_sanity_val_steps=0,
    callbacks=[checkpoint_callback]
)

trainer.fit(sentiment_pl)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                          | Params
------------------------------------------------------------
0 | model     | BertForSequenceClassification | 167 M 
1 | loss_func | CrossEntropyLoss              | 0     
-----------------------------

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Val loss: 0.32891377806663513
Val f1: 0.6737083312425782
Train loss: 0.35420048236846924
Train f1: 0.6251215586409684


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Val loss: 0.4246973991394043
Val f1: 0.6742362660170882
Train loss: 0.13005143404006958
Train f1: 0.8785016894633761


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Val loss: 0.5860643982887268
Val f1: 0.6058249208934143
Train loss: 0.0751800686120987
Train f1: 0.9329830337696376


In [79]:
sentiment_pl.stage = 'test'
sentiment_pl.eval()

results = trainer.test(sentiment_pl, ckpt_path='best')
with open('/home/jovyan/notebooks/vk/sentiment_original.txt', 'w') as f:
    print(results, file=f)

Restoring states from the checkpoint path at /home/jovyan/notebooks/anonymization/checkpoints_ru_sentiment/epoch=1-step=1972.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/jovyan/notebooks/anonymization/checkpoints_ru_sentiment/epoch=1-step=1972.ckpt
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

Test loss: 0.30249857902526855
Test f1: 0.6790496749428991
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       avg_test_f1          0.6790496749428991
      avg_test_loss         0.30249857902526855
         test_f1            0.6790496749428991
        test_loss           0.30249857902526855
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [83]:
anon_train_dataset = SentimentDataset(train_sentiment.anonymized_text.tolist(), train_sentiment.target.tolist(), sentiment_tokenizer)
anon_test_dataset = SentimentDataset(test_sentiment.anonymized_text.tolist(), test_sentiment.target.tolist(), sentiment_tokenizer)
sentiment_pl = LitSentiment(sentiment_model_init, anon_train_dataset, anon_test_dataset, 2e-5)
sentiment_pl.train()

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=chechpoint_path_sentiment, save_top_k=2, monitor="val_loss")

trainer = Trainer(
    max_epochs=5,
    num_nodes=1,
    num_sanity_val_steps=0,
    callbacks=[checkpoint_callback]
)

trainer.fit(sentiment_pl)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/jovyan/notebooks/anonymization/lightning_logs
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/jovyan/notebooks/anonymization/checkpoints_ru_sentiment exists and is not empty.
  rank_zer

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Val loss: 0.34638315439224243
Val f1: 0.6643685082041251
Train loss: 0.3769320845603943
Train f1: 0.6378467341674641


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Val loss: 0.4136177897453308
Val f1: 0.6738311916394112
Train loss: 0.24240928888320923
Train f1: 0.7623966605443407


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Val loss: 0.44836264848709106
Val f1: 0.6729723853011527
Train loss: 0.14990410208702087
Train f1: 0.8664648723517047


In [84]:
sentiment_pl.stage = 'test'
sentiment_pl.eval()

results = trainer.test(sentiment_pl, ckpt_path='best')
with open('/home/jovyan/notebooks/vk/sentiment_anonymized.txt', 'w') as f:
    print(results, file=f)

Restoring states from the checkpoint path at /home/jovyan/notebooks/anonymization/checkpoints_ru_sentiment/epoch=1-step=1972-v2.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/jovyan/notebooks/anonymization/checkpoints_ru_sentiment/epoch=1-step=1972-v2.ckpt
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

Test loss: 0.3402031660079956
Test f1: 0.6381162984859087
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       avg_test_f1          0.6381162984859087
      avg_test_loss         0.3402031660079956
         test_f1            0.6381162984859087
        test_loss           0.3402031660079956
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
